In [3]:
def process_data_and_save(input_files, output_files, group_column):
    """
    Processes multiple DataFrames by sorting and adding rank columns, then saves the CSV files.

    Parameters:
    - input_files: list of str - Paths to the input CSV files.
    - output_files: list of str - Paths to the output CSV files.
    - group_column: str - The column to group by for sorting and ranking.
    """
    for input_file, output_file in zip(input_files, output_files):
        # Load the data
        df = pd.read_csv(input_file)

        # Clean column names by removing leading and trailing whitespace
        df.columns = df.columns.str.strip()

        # Determine sorting columns based on the input file
        if 'sec1' in input_file:
            sort_columns = ['profitability_per_txn', 'cost_per_txn', 'reversal_per_txn']
            rank_column = 'Rank_1'
            ascending_order = [False, True, True]  # Descending for profitability_per_txn, ascending for cost_per_txn and reversal_per_txn
        elif 'sec2' in input_file:
            sort_columns = ['success_percentage', 'failure_percentage']
            rank_column = 'Rank_2'
            ascending_order = [False, True]  # Descending for success_percentage, ascending for failure_percentage
        elif 'sec3' in input_file:
            sort_columns = ['less_15m', 'greater_15m']
            rank_column = 'Rank_3'
            ascending_order = [False, True]  # Descending for less_15m, ascending for greater_15m
        else:
            print(f"Unknown file format for {input_file}. Skipping.")
            continue

        # Sort the DataFrame based on the specified columns
        sorted_df = df.sort_values(by=[group_column] + sort_columns, ascending=[True] + ascending_order)

        # Add a rank column within each group defined by 'group_column'
        sorted_df[rank_column] = sorted_df.groupby(group_column).cumcount() + 1

        # Save the final DataFrame to a new CSV file
        sorted_df.to_csv(output_file, index=False)
        print(f"\nDataFrame processed and saved to '{output_file}'")
    
def merge_and_sort_csv_files(file1_path, file2_path, file3_path,file1_cols,file2_cols,file3_cols):
    """
    Merges three CSV files based on destination_country and destination_partner, 
    calculates total and average ranks, sorts the result, and returns the sorted DataFrame.

    Parameters:
    - file1_path: str - Path to the first CSV file.
    - file2_path: str - Path to the second CSV file.
    - file3_path: str - Path to the third CSV file.
    - columns_to_drop: list - List of columns to drop from the final DataFrame.
    
    Returns:
    - DataFrame - The sorted DataFrame after merging and processing.
    """
    # Read the CSV files into DataFrames
    df1 = pd.read_csv(file1_path)
    df2 = pd.read_csv(file2_path)
    df3 = pd.read_csv(file3_path)

    # Merge df1 with df2 on destination_country and destination_partner
    merged_df = pd.merge(df1, df2, on=['destination_country', 'destination_partner'], how='left', suffixes=('', '_file2'))
    
    # Merge the result with df3 on destination_country and destination_partner
    merged_df = pd.merge(merged_df, df3, on=['destination_country', 'destination_partner'], how='left', suffixes=('', '_file3'))

    # Drop duplicate columns if they exist
    merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

    # Create a list of columns excluding the rank columns
    non_rank_columns = [col for col in file1_cols + file2_cols + file3_cols if 'Rank' not in col]
    rank_columns =  [col for col in file1_cols + file2_cols + file3_cols if 'Rank' in col]

    # Select the specific columns from the merged DataFrame
    selected_columns = non_rank_columns + rank_columns
    sorted_df = merged_df[selected_columns].copy()  # Create a copy to avoid SettingWithCopyWarning

    # Add a new column for Total_Rank, summing Rank_1, Rank_2, and Rank_3
    sorted_df.loc[:, 'Total_Rank'] = sorted_df[['Rank_1', 'Rank_2', 'Rank_3']].fillna(0).sum(axis=1)

    # Add a new column for Average_Rank
    sorted_df.loc[:, 'Average_Rank'] = (sorted_df['Total_Rank'] / 3).round(2)  # Calculate average and round to 2 decimal places

    return sorted_df  # Return the sorted DataFrame for further processing
    
def swap_rows_based_on_columns(df, columns_to_compare):
    """
    Swaps rows in the DataFrame based on specified columns.

    Parameters:
    - df: DataFrame - The DataFrame to modify.
    - columns_to_compare: list - List of columns to compare for swapping.
    
    Returns:
    - DataFrame - The modified DataFrame with swapped rows and specified columns dropped
    """
    # Group by 'destination_country' and 'Average_Rank'
    grouped = df.groupby(['destination_country', 'Average_Rank'])

    # Create a new DataFrame to hold the modified rows
    modified_rows = []

    # Iterate through each group
    for name, group in grouped:
        if len(group) > 1:  # Only consider groups with more than one row
            # Convert the group to a DataFrame for easier manipulation
            group_df = group.copy()

            # Compare the specified columns and swap rows if necessary
            for i in range(len(group_df)):
                for j in range(i + 1, len(group_df)):
                    # Compare the values of the specified columns
                    if any(group_df.iloc[i][col] > group_df.iloc[j][col] for col in columns_to_compare):
                        # Swap the rows
                        group_df.iloc[[i, j]] = group_df.iloc[[j, i]].values

            # Append the modified group to the list
            modified_rows.append(group_df)

        else:
            # If there's only one row, just append it as is
            modified_rows.append(group)

    # Concatenate all modified groups into a single DataFrame
    modified_df = pd.concat(modified_rows)

    return modified_df  # Return the modified DataFrame

# Example usage
if __name__ == "__main__":
    # Specify the input files and output files for ranking
    input_files = ['PART 2 sec1.csv', 'PART 2 sec2.csv', 'PART 2 sec3.csv']
    output_files = ['ranked_data1.csv', 'ranked_data2.csv', 'ranked_data3.csv']

    # Process the data and save to CSV
    process_data_and_save(input_files, output_files, 'destination_country')

    file1_cols = ['destination_country', 'destination_partner', 'profitability_per_txn', 'cost_per_txn', 'reversal_per_txn', 'Rank_1']  # Select specific columns from file1
    file2_cols = ['success_percentage', 'failure_percentage', 'Rank_2']  # Select specific columns from file2
    file3_cols = ['less_15m', 'greater_15m', 'Rank_3']  # Select specific columns from file3


    # Specify the input files for merging and sorting
    file1 = 'ranked_data1.csv'  
    file2 = 'ranked_data2.csv'  
    file3 = 'ranked_data3.csv'  
    sorted_output_file = 'sorted_output.csv'  # Output file for sorted data
    modified_output_file = 'Destination_partner_ranking.csv'  # Output file for modified data


    # Call the function to merge and sort the CSV files
    sorted_df = merge_and_sort_csv_files(file1, file2, file3,  file1_cols,file2_cols,file3_cols)

    # Specify the columns to compare for swapping
    columns_to_compare = ['profitability_per_txn', 'success_percentage', 'less_15m']  # Replace with actual column names you want to compare

    
    # Call the function to swap rows based on the specified columns
    modified_df = swap_rows_based_on_columns(sorted_df, columns_to_compare)
    
    # Save the modified Data Frame to a new CSV file
    modified_df.to_csv(modified_output_file, index=False)

    # Print a message indicating that the modified file has been created successfully
    print(f"Modified file '{modified_output_file}' has been created successfully.")


DataFrame processed and saved to 'ranked_data1.csv'

DataFrame processed and saved to 'ranked_data2.csv'

DataFrame processed and saved to 'ranked_data3.csv'
Modified file 'Destination_partner_ranking.csv' has been created successfully.
